Let's represent the firewall as a list of cycles. For empty spaces, we can have a cycle of -1.

In [1]:
import itertools as it
import re


First parse the input:

In [2]:
testInput_str='''
0: 3
1: 2
4: 4
6: 4
'''

In [3]:
testInput_dict={int(nl.split(':')[0].strip()): int(nl.split(':')[1].strip())
                for nl in testInput_str.split('\n') if len(nl.split(':'))==2}
testInput_dict

{0: 3, 1: 2, 4: 4, 6: 4}

Now, for each scanner, want a cycle that goes forward and backward over the length of the range:

In [4]:
def make_scanner(range_i):
    '''Returns a cycling iterator over the range
       of the scanner.
    '''
    # Don't know if this can take a value of 1
    assert range_i != 1
    # And it can't take a negative value:
    assert range_i>=0
    
    # If range_i is zero, return a cycle of -1
    if range_i==0:
        return it.cycle([-1])
    
    else:
        scannerPathBackward_ls=list(range(range_i))[1:-1]
        scannerPathBackward_ls.reverse()
        return it.cycle(list(range(range_i))+scannerPathBackward_ls)

list(it.islice(make_scanner(4), 20))

[0, 1, 2, 3, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1]

and now we can represent the firewall as a list of the scanners:

In [5]:
scanners_ls=[make_scanner(testInput_dict.get(i, 0)) for i in range(max(testInput_dict)+1)]

scanners_ls
                          

Now, to construct the whole firewall, we can construct a zip of all the scanners. So for the test case, we'll have:

In [6]:
firewall_zip=zip(*scanners_ls)

and we can represent the state by a pair of the current location of the packet and the scanners. We can use the first argument to limit the states counted:

In [7]:
state_zip=zip(range(max(testInput_dict)+1), firewall_zip)

Now we can calculate the severity of each stage. We'll need to pass the input information, to get the range. 

In [8]:
def severity(stateIn, input_dict):
    '''
    Return the severity of the current position, given an
    input state and the input list
    '''
    packetPos_i=stateIn[0]
    firewallState_ls=stateIn[1]
    if firewallState_ls[packetPos_i]==0:
        return packetPos_i*input_dict[packetPos_i]
    else:
        return 0

In [9]:
[severity(s, testInput_dict) for s in state_zip]

[0, 0, 0, 0, 0, 0, 24]

Good. So the final output is the sum of the severities:

In [10]:
sum(_)

24

Now do it with the puzzle input:

In [11]:
with open('data/day13.txt') as fIn:
    puzzleInput_str=fIn.read()

In [12]:
puzzleInput_dict={int(nl.split(':')[0].strip()): int(nl.split(':')[1].strip())
                  for nl in puzzleInput_str.split('\n') if len(nl.split(':'))==2}
puzzleInput_dict

{0: 5,
 1: 2,
 2: 3,
 4: 4,
 6: 6,
 8: 4,
 10: 8,
 12: 6,
 14: 6,
 16: 14,
 18: 6,
 20: 8,
 22: 8,
 24: 10,
 26: 8,
 28: 8,
 30: 10,
 32: 8,
 34: 12,
 36: 9,
 38: 20,
 40: 12,
 42: 12,
 44: 12,
 46: 12,
 48: 12,
 50: 12,
 52: 12,
 54: 12,
 56: 14,
 58: 14,
 60: 14,
 62: 20,
 64: 14,
 66: 14,
 70: 14,
 72: 14,
 74: 14,
 76: 14,
 78: 14,
 80: 12,
 90: 30,
 92: 17,
 94: 18}

In [13]:
scanners_ls=[make_scanner(puzzleInput_dict.get(i, 0)) for i in range(max(puzzleInput_dict)+1)]

state_zip=zip(range(max(puzzleInput_dict)+1), zip(*scanners_ls))

state_zip

In [14]:
severity_ls=[severity(s, puzzleInput_dict) for s in state_zip]
severity_ls

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 224,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 760,
 0,
 0,
 0,
 0,
 0,
 528,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1092,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
sum(severity_ls)

2604